In [2]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score, average_precision_score

# 1. 读取边列表，并保留权重
df_edges = pd.read_csv("ingredient_cooccur_graph.csv")
edges = list(zip(df_edges["source"], df_edges["target"]))
weights = {
    (u, v): w
    for u, v, w in zip(df_edges["source"], df_edges["target"], df_edges["weight"])
}

# 2. 拆分正例
e_train, e_tmp = train_test_split(edges, test_size=0.30, random_state=42)
e_val, e_test = train_test_split(e_tmp, test_size=0.50, random_state=42)

# 3. 构建训练图（加权）
G_train = nx.Graph()
G_train.add_nodes_from(set(df_edges["source"]) | set(df_edges["target"]))
G_train.add_weighted_edges_from([
    (u, v, weights.get((u, v), weights.get((v, u), 1)))
    for u, v in e_train
])
nodes = list(G_train.nodes())

# 4. 计算 cooc 和 PMI 所需的统计量
train_cooc = nx.get_edge_attributes(G_train, 'weight')
total_pairs = sum(train_cooc.values())
marginal = {}
for (u, v), w in train_cooc.items():
    marginal[u] = marginal.get(u, 0) + w
    marginal[v] = marginal.get(v, 0) + w
pmi = {}
for (u, v), w in train_cooc.items():
    pi, pj = marginal[u] / total_pairs, marginal[v] / total_pairs
    pij = w / total_pairs
    pmi[(u, v)] = np.log(pij / (pi * pj) + 1e-9)

# 5. 构建矩阵 A 并做 Truncated SVD 嵌入
idx_map = { node: i for i, node in enumerate(nodes) }
A = np.zeros((len(nodes), len(nodes)))
for (u, v), w in train_cooc.items():
    i, j = idx_map[u], idx_map[v]
    A[i, j] = w
    A[j, i] = w
svd = TruncatedSVD(n_components=20, random_state=42)
emb = normalize(svd.fit_transform(A))

def svd_cos(u, v):
    return float(np.dot(emb[idx_map[u]], emb[idx_map[v]]))

# 6. 定义 CN 和 AA
cn  = lambda u, v: len(list(nx.common_neighbors(G_train, u, v)))
aa_index = { (u, v): s for u, v, s in nx.adamic_adar_index(G_train) }

# 7. 负采样（测试集）
def sample_neg_edges(G, n, forbidden):
    negs = set()
    while len(negs) < n:
        u, v = np.random.choice(nodes, 2, replace=False)
        if (u, v) not in forbidden and (v, u) not in forbidden and not G.has_edge(u, v):
            negs.add((u, v))
    return list(negs)

forbidden = set(edges)
e_test_neg = sample_neg_edges(G_train, len(e_test), forbidden)

# 8. 在原始测试集上评估五种方法
test_pairs_orig  = e_test + e_test_neg
test_labels_orig = [1]*len(e_test) + [0]*len(e_test_neg)

scores_orig = {
    'Cooc': [train_cooc.get(p, train_cooc.get((p[1],p[0]), 0)) for p in test_pairs_orig],
    'PMI':  [pmi.get(p,  pmi.get((p[1],p[0]), 0))               for p in test_pairs_orig],
    'CN':   [cn(u, v)                                          for u, v in test_pairs_orig],
    'AA':   [aa_index.get(p, aa_index.get((p[1],p[0]), 0))    for p in test_pairs_orig],
    'SVD':  [svd_cos(u, v)                                     for u, v in test_pairs_orig],
}

print("Original test metrics:")
for name, sc in scores_orig.items():
    auc = roc_auc_score(test_labels_orig, sc)
    ap  = average_precision_score(test_labels_orig, sc)
    print(f"  {name:4s} → AUC: {auc:.4f}, AP: {ap:.4f}")

# 9. Rare 子集测试（度在前 25% 的节点）
deg = dict(G_train.degree())
threshold = np.percentile(list(deg.values()), 25)
rare_pos = [e for e in e_test     if deg[e[0]] <= threshold and deg[e[1]] <= threshold]
rare_neg = [e for e in e_test_neg if deg[e[0]] <= threshold and deg[e[1]] <= threshold]

test_pairs_rare  = rare_pos + rare_neg
test_labels_rare = [1]*len(rare_pos) + [0]*len(rare_neg)

scores_rare = {
    'Cooc': [train_cooc.get(p, train_cooc.get((p[1],p[0]), 0)) for p in test_pairs_rare],
    'PMI':  [pmi.get(p,  pmi.get((p[1],p[0]), 0))               for p in test_pairs_rare],
    'CN':   [cn(u, v)                                          for u, v in test_pairs_rare],
    'AA':   [aa_index.get(p, aa_index.get((p[1],p[0]), 0))    for p in test_pairs_rare],
    'SVD':  [svd_cos(u, v)                                     for u, v in test_pairs_rare],
}

print("\nRare test metrics:")
for name, sc in scores_rare.items():
    auc = roc_auc_score(test_labels_rare, sc)
    ap  = average_precision_score(test_labels_rare, sc)
    print(f"  {name:4s} → AUC: {auc:.4f}, AP: {ap:.4f}")


Original test metrics:
  Cooc → AUC: 0.5000, AP: 0.5000
  PMI  → AUC: 0.5000, AP: 0.5000
  CN   → AUC: 0.9335, AP: 0.9387
  AA   → AUC: 0.9347, AP: 0.9409
  SVD  → AUC: 0.7501, AP: 0.7180

Rare test metrics:
  Cooc → AUC: 0.5000, AP: 0.0029
  PMI  → AUC: 0.5000, AP: 0.0029
  CN   → AUC: 0.9895, AP: 0.1422
  AA   → AUC: 0.9942, AP: 0.2667
  SVD  → AUC: 0.9769, AP: 0.5294


GCN + Cos

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score

# — 1. Load edge list
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
df_edges = pd.read_csv("ingredient_cooccur_graph.csv")
edges     = list(zip(df_edges["source"], df_edges["target"]))
weights   = {(u, v): w for u, v, w in zip(df_edges["source"], df_edges["target"], df_edges["weight"]) }

# — 2. Load features
df_feat    = pd.read_csv("feature_matrix.csv", index_col="node")

# —— 保持和训练图完全一致的节点顺序
nodes_full = sorted(set(df_edges["source"]) | set(df_edges["target"]))
df_feat    = df_feat.loc[nodes_full]

# —— 构造张量
X          = torch.tensor(df_feat.values.astype(np.float32), device=device)

# —— 建立索引映射
idx_map    = {n: i for i, n in enumerate(nodes_full)}


# — 3. Split edges
e_train, e_tmp  = train_test_split(edges, test_size=0.30, random_state=42)
e_val,   e_test = train_test_split(e_tmp,   test_size=0.50, random_state=42)

# — 4. Build train graph
G_train = nx.Graph()
G_train.add_nodes_from(nodes_full)
G_train.add_weighted_edges_from([
    (u, v, weights.get((u, v), weights.get((v, u), 1.0)))
    for u, v in e_train
])

# compute degree for rare-split (you can use weighted degree if preferred)
deg = dict(G_train.degree())

# — 5. Negative sampling helper
def sample_neg(n, forbidden, G):
    negs = set()
    while len(negs) < n:
        u, v = np.random.choice(nodes_full, 2, replace=False)
        if ((u, v) not in forbidden and (v, u) not in forbidden and not G.has_edge(u, v)):
            negs.add((u, v))
    return list(negs)

forbidden   = set(edges)
e_train_neg = sample_neg(len(e_train), forbidden, G_train)
e_val_neg   = sample_neg(len(e_val),   forbidden, G_train)
e_test_neg  = sample_neg(len(e_test),  forbidden, G_train)

# — 6. Build normalized adjacency
N = len(nodes_full)
A = np.zeros((N, N), dtype=np.float32)
for u, v in e_train:
    i, j = idx_map[u], idx_map[v]
    w    = weights.get((u, v), weights.get((v, u), 1.0))
    A[i, j] = A[j, i] = w

deg_array = A.sum(axis=1)
D_inv_s   = np.diag(1.0 / np.sqrt(deg_array + 1e-9))
A_norm    = torch.tensor(D_inv_s @ A @ D_inv_s, device=device)

# — 7. Define GCN
class SimpleGCN(nn.Module):
    def __init__(self, in_dim, hid_dim=64):
        super().__init__()
        self.lin1 = nn.Linear(in_dim, hid_dim)
        self.lin2 = nn.Linear(hid_dim, hid_dim)
    def encode(self, x, A):
        h = A @ x
        h = F.relu(self.lin1(h))
        h = A @ h
        h = F.relu(self.lin2(h))
        return h

# Instantiate
model     = SimpleGCN(in_dim=X.shape[1], hid_dim=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3, weight_decay=1e-4)
criterion = nn.BCEWithLogitsLoss()

# Prepare data
train_pairs = e_train + e_train_neg
train_lbls  = torch.cat([torch.ones(len(e_train)), torch.zeros(len(e_train_neg))], dim=0).to(device)
val_pairs   = e_val   + e_val_neg
val_lbls    = torch.cat([torch.ones(len(e_val)),   torch.zeros(len(e_val_neg))],   dim=0).to(device)

# — 8. Train with Early Stopping
best_val_loss = float('inf')
patience      = 10
patience_cnt  = 0
best_state    = None

for epoch in range(1, 51):
    model.train()
    optimizer.zero_grad()
    Z = model.encode(X, A_norm)
    logits = torch.stack([
        (Z[idx_map[u]] * Z[idx_map[v]]).sum()
        for u, v in train_pairs
    ])
    loss = criterion(logits, train_lbls)
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        Z_val      = model.encode(X, A_norm)
        logits_val = torch.stack([
            (Z_val[idx_map[u]] * Z_val[idx_map[v]]).sum()
            for u, v in val_pairs
        ])
        val_loss = criterion(logits_val, val_lbls)

    print(f"Epoch {epoch:2d} | Train Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f}")

    if val_loss.item() < best_val_loss:
        best_val_loss = val_loss.item()
        best_state    = model.state_dict()
        patience_cnt  = 0
    else:
        patience_cnt += 1
        if patience_cnt >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

if best_state is not None:
    model.load_state_dict(best_state)

# — 9. Test evaluation on two splits
model.eval()
with torch.no_grad():
    Z_np = model.encode(X, A_norm).cpu().numpy()

# -- Original test set
orig_pairs = e_test + e_test_neg
orig_lbls  = np.array([1]*len(e_test) + [0]*len(e_test_neg))
orig_scores = [
    float(
        np.dot(Z_np[idx_map[u]], Z_np[idx_map[v]]) /
        (np.linalg.norm(Z_np[idx_map[u]]) * np.linalg.norm(Z_np[idx_map[v]]) + 1e-9)
    )
    for u, v in orig_pairs
]
orig_auc = roc_auc_score(orig_lbls, orig_scores)
orig_ap  = average_precision_score(orig_lbls, orig_scores)
print(f"\nGCN + Cosine on Original test → AUC: {orig_auc:.4f}, AP: {orig_ap:.4f}")

# -- Rare test set (degree ≤ 25th percentile)
threshold = np.percentile(list(deg.values()), 25)
rare_pos  = [e for e in e_test     if deg[e[0]] <= threshold and deg[e[1]] <= threshold]
rare_neg  = [e for e in e_test_neg if deg[e[0]] <= threshold and deg[e[1]] <= threshold]
rare_pairs = rare_pos + rare_neg
rare_lbls  = np.array([1]*len(rare_pos) + [0]*len(rare_neg))
rare_scores = [
    float(
        np.dot(Z_np[idx_map[u]], Z_np[idx_map[v]]) /
        (np.linalg.norm(Z_np[idx_map[u]]) * np.linalg.norm(Z_np[idx_map[v]]) + 1e-9)
    )
    for u, v in rare_pairs
]
rare_auc = roc_auc_score(rare_lbls, rare_scores)
rare_ap  = average_precision_score(rare_lbls, rare_scores)
print(f"GCN + Cosine on Rare   test → AUC: {rare_auc:.4f}, AP: {rare_ap:.4f}")


Using device: cuda
Epoch  1 | Train Loss: 0.6960 | Val Loss: 0.6858
Epoch  2 | Train Loss: 0.6859 | Val Loss: 0.6712
Epoch  3 | Train Loss: 0.6713 | Val Loss: 0.6504
Epoch  4 | Train Loss: 0.6505 | Val Loss: 0.6250
Epoch  5 | Train Loss: 0.6254 | Val Loss: 0.5995
Epoch  6 | Train Loss: 0.6000 | Val Loss: 0.5792
Epoch  7 | Train Loss: 0.5799 | Val Loss: 0.5672
Epoch  8 | Train Loss: 0.5683 | Val Loss: 0.5638
Epoch  9 | Train Loss: 0.5653 | Val Loss: 0.5645
Epoch 10 | Train Loss: 0.5662 | Val Loss: 0.5627
Epoch 11 | Train Loss: 0.5645 | Val Loss: 0.5559
Epoch 12 | Train Loss: 0.5574 | Val Loss: 0.5456
Epoch 13 | Train Loss: 0.5469 | Val Loss: 0.5350
Epoch 14 | Train Loss: 0.5360 | Val Loss: 0.5266
Epoch 15 | Train Loss: 0.5273 | Val Loss: 0.5213
Epoch 16 | Train Loss: 0.5216 | Val Loss: 0.5182
Epoch 17 | Train Loss: 0.5182 | Val Loss: 0.5154
Epoch 18 | Train Loss: 0.5152 | Val Loss: 0.5120
Epoch 19 | Train Loss: 0.5116 | Val Loss: 0.5079
Epoch 20 | Train Loss: 0.5074 | Val Loss: 0.5036
E

GCN + MLP

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score

# — 1. Load edge list
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
df_edges = pd.read_csv("ingredient_cooccur_graph.csv")
edges     = list(zip(df_edges["source"], df_edges["target"]))
weights   = {(u, v): w for u, v, w in zip(df_edges["source"], df_edges["target"], df_edges["weight"]) }

# — 2. Load features
df_feat    = pd.read_csv("feature_matrix.csv", index_col="node")

# —— 保持和训练图完全一致的节点顺序
nodes_full = sorted(set(df_edges["source"]) | set(df_edges["target"]))
df_feat    = df_feat.loc[nodes_full]

# —— 构造张量
X          = torch.tensor(df_feat.values.astype(np.float32), device=device)

# —— 建立索引映射
idx_map    = {n: i for i, n in enumerate(nodes_full)}


# — 3. Split edges
e_train, e_tmp  = train_test_split(edges, test_size=0.30, random_state=42)
e_val,   e_test = train_test_split(e_tmp,   test_size=0.50, random_state=42)

# — 4. Build train graph
G_train = nx.Graph()
G_train.add_nodes_from(nodes_full)
G_train.add_weighted_edges_from([
    (u, v, weights.get((u, v), weights.get((v, u), 1.0)))
    for u, v in e_train
])

# compute degree for rare-split (you can use weighted degree if preferred)
deg = dict(G_train.degree())

# — 5. Negative sampling helper
def sample_neg(n, forbidden, G):
    negs = set()
    while len(negs) < n:
        u, v = np.random.choice(nodes_full, 2, replace=False)
        if ((u, v) not in forbidden and (v, u) not in forbidden and not G.has_edge(u, v)):
            negs.add((u, v))
    return list(negs)

forbidden   = set(edges)
e_train_neg = sample_neg(len(e_train), forbidden, G_train)
e_val_neg   = sample_neg(len(e_val),   forbidden, G_train)
e_test_neg  = sample_neg(len(e_test),  forbidden, G_train)

# — 6. Build normalized adjacency
N = len(nodes_full)
A = np.zeros((N, N), dtype=np.float32)
for u, v in e_train:
    i, j = idx_map[u], idx_map[v]
    w    = weights.get((u, v), weights.get((v, u), 1.0))
    A[i, j] = A[j, i] = w

deg_array = A.sum(axis=1)
D_inv_s   = np.diag(1.0 / np.sqrt(deg_array + 1e-9))
A_norm    = torch.tensor(D_inv_s @ A @ D_inv_s, device=device)

# — 7. Define GCN
class SimpleGCN(nn.Module):
    def __init__(self, in_dim, hid_dim=64):
        super().__init__()
        self.lin1 = nn.Linear(in_dim, hid_dim)
        self.lin2 = nn.Linear(hid_dim, hid_dim)
    def encode(self, x, A):
        h = A @ x
        h = F.relu(self.lin1(h))
        h = A @ h
        h = F.relu(self.lin2(h))
        return h

hid_dim = 64
decoder = nn.Sequential(
    nn.Linear(hid_dim*2, hid_dim),
    nn.ReLU(),
    nn.Linear(hid_dim, 1)
).to(device)

# Instantiate
model     = SimpleGCN(in_dim=X.shape[1], hid_dim=64).to(device)
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(decoder.parameters()),
    lr=5e-3, weight_decay=1e-4
)
criterion = nn.BCEWithLogitsLoss()

# Prepare data
train_pairs = e_train + e_train_neg
train_lbls  = torch.cat([torch.ones(len(e_train)), torch.zeros(len(e_train_neg))], dim=0).to(device)
val_pairs   = e_val   + e_val_neg
val_lbls    = torch.cat([torch.ones(len(e_val)),   torch.zeros(len(e_val_neg))],   dim=0).to(device)

# — 8. Train with Early Stopping
best_val_loss = float('inf')
patience      = 10
patience_cnt  = 0
best_state    = None



for epoch in range(1, 51):
    model.train()
    decoder.train()
    optimizer.zero_grad()

    Z = model.encode(X, A_norm)  # (N, hid_dim)

    # 构造训练对的拼接向量
    emb_pairs = torch.stack([
        torch.cat([Z[idx_map[u]], Z[idx_map[v]]], dim=0)
        for u, v in train_pairs
    ])  # (num_pairs, hid_dim*2)

    logits = decoder(emb_pairs).squeeze()        # (num_pairs,)
    loss   = criterion(logits, train_lbls)       # BCEWithLogitsLoss

    loss.backward()
    optimizer.step()

    # 验证同理
    model.eval()
    decoder.eval()
    with torch.no_grad():
        Z_val = model.encode(X, A_norm)
        emb_val = torch.stack([
            torch.cat([Z_val[idx_map[u]], Z_val[idx_map[v]]], dim=0)
            for u, v in val_pairs
        ])
        logits_val = decoder(emb_val).squeeze()
        val_loss   = criterion(logits_val, val_lbls)

    print(f"Epoch {epoch:2d} | Train Loss: {loss:.4f} | Val Loss: {val_loss:.4f}")

    if val_loss.item() < best_val_loss:
        best_val_loss = val_loss.item()
        best_state    = model.state_dict()
        patience_cnt  = 0
    else:
        patience_cnt += 1
        if patience_cnt >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

if best_state is not None:
    model.load_state_dict(best_state)

# — 9. Test evaluation on two splits
orig_pairs = e_test + e_test_neg
orig_lbls  = np.array([1]*len(e_test) + [0]*len(e_test_neg))

threshold = np.percentile(list(deg.values()), 25)
rare_pos  = [e for e in e_test     if deg[e[0]] <= threshold and deg[e[1]] <= threshold]
rare_neg  = [e for e in e_test_neg if deg[e[0]] <= threshold and deg[e[1]] <= threshold]
rare_pairs = rare_pos + rare_neg
rare_lbls  = np.array([1]*len(rare_pos) + [0]*len(rare_neg))

def precision_at_k(labels: np.ndarray, scores: np.ndarray, k: int) -> float:
    idx_desc = np.argsort(scores)[::-1][:k]
    return labels[idx_desc].sum() / float(k)

model.eval()
decoder.eval()
with torch.no_grad():
    Z_final = model.encode(X, A_norm)

    # 原始测试集
    emb_test = torch.stack([
        torch.cat([Z_final[idx_map[u]], Z_final[idx_map[v]]], dim=0)
        for u, v in orig_pairs
    ])
    logits_test = decoder(emb_test).squeeze()
    probs_test  = torch.sigmoid(logits_test).cpu().numpy()
    orig_auc = roc_auc_score(orig_lbls, probs_test)
    orig_ap  = average_precision_score(orig_lbls, probs_test)
    print(f"\nGCN + MLP on Original test → AUC: {orig_auc:.4f}, AP: {orig_ap:.4f}")

    for K in [10, 20, 50]:
        p_at_k = precision_at_k(orig_lbls, probs_test, K)
        print(f"Precision@{K} on Original test → {p_at_k:.4f}")

    # 稀有子集
    emb_rare = torch.stack([
        torch.cat([Z_final[idx_map[u]], Z_final[idx_map[v]]], dim=0)
        for u, v in rare_pairs
    ])
    logits_rare = decoder(emb_rare).squeeze()
    probs_rare  = torch.sigmoid(logits_rare).cpu().numpy()
    rare_auc = roc_auc_score(rare_lbls, probs_rare)
    rare_ap  = average_precision_score(rare_lbls, probs_rare)
    print(f"GCN + MLP on Rare   test → AUC: {rare_auc:.4f}, AP: {rare_ap:.4f}")

    for K in [10, 20, 50]:
        p_at_k_rare = precision_at_k(rare_lbls, probs_rare, K)
        print(f"Precision@{K} on Rare test     → {p_at_k_rare:.4f}")


Using device: cuda
Epoch  1 | Train Loss: 0.6943 | Val Loss: 0.6915
Epoch  2 | Train Loss: 0.6916 | Val Loss: 0.6886
Epoch  3 | Train Loss: 0.6887 | Val Loss: 0.6841
Epoch  4 | Train Loss: 0.6843 | Val Loss: 0.6773
Epoch  5 | Train Loss: 0.6776 | Val Loss: 0.6673
Epoch  6 | Train Loss: 0.6676 | Val Loss: 0.6531
Epoch  7 | Train Loss: 0.6537 | Val Loss: 0.6342
Epoch  8 | Train Loss: 0.6350 | Val Loss: 0.6105
Epoch  9 | Train Loss: 0.6117 | Val Loss: 0.5826
Epoch 10 | Train Loss: 0.5842 | Val Loss: 0.5522
Epoch 11 | Train Loss: 0.5544 | Val Loss: 0.5220
Epoch 12 | Train Loss: 0.5250 | Val Loss: 0.4950
Epoch 13 | Train Loss: 0.4989 | Val Loss: 0.4734
Epoch 14 | Train Loss: 0.4783 | Val Loss: 0.4568
Epoch 15 | Train Loss: 0.4630 | Val Loss: 0.4452
Epoch 16 | Train Loss: 0.4528 | Val Loss: 0.4381
Epoch 17 | Train Loss: 0.4471 | Val Loss: 0.4296
Epoch 18 | Train Loss: 0.4394 | Val Loss: 0.4167
Epoch 19 | Train Loss: 0.4266 | Val Loss: 0.4006
Epoch 20 | Train Loss: 0.4100 | Val Loss: 0.3834
E